### Resample images from MSD and save as h5 files

In [2]:
import os
import os.path as osp
import SimpleITK as sitk
from glob import glob
from tqdm import tqdm
import nibabel as nib
import numpy as np
from monai.transforms import *

/home/VANDERBILT/liuh26/anaconda3/envs/seg/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def resample(data_dict, out_spacing, save_dir):
    
    return Compose([
        LoadImaged(keys=['image', 'label']),
        
        AddChanneld(keys=["image", "label"]),
        
        Spacingd(
            keys=['image', 'label'], 
            pixdim=out_spacing, 
            mode=("bilinear", "nearest")),
        
        SaveImaged(
            keys=['image'], 
            output_dir=osp.join(save_dir, 'imagesTr_res'), 
            output_postfix='', 
            output_ext='.nii.gz', 
            resample=False,
            separate_folder=False,
            print_log=False),
        
        SaveImaged(
            keys=['label'], 
            output_dir=osp.join(save_dir, 'labelsTr_res'), 
            output_postfix='', 
            output_ext='.nii.gz', 
            resample=False,
            separate_folder=False,
            print_log=False),
        
    ])(data_dict)


In [4]:
# data_dirs = ['../../data/Task09_Spleen', 
#              '../../data/Task03_Liver', 
#              '../../data/Task07_Pancreas',
#              '../../data/Task08_HepaticVessel']

data_dirs = ['../../data/Task03_Liver', 
             '../../data/Task07_Pancreas',
             '../../data/Task08_HepaticVessel']


for data_dir in data_dirs:
    img_paths = glob(osp.join(data_dir, 'imagesTr') + '/*.*')

    with tqdm(total=len(img_paths)) as pbar:
        for path in img_paths:
            msk_path = osp.join(data_dir, 'labelsTr', osp.basename(path))
            data_dict = {'image': path, 'label': msk_path}
            resample(data_dict, (1.5, 1.5, 2.0), data_dir)
            pbar.update(1)
        
        
        

100%|█████████████████████████████████████████████| 303/303 [20:53<00:00,  4.14s/it]


In [5]:
# data_dirs = ['../../data/Task09_Spleen', 
#              '../../data/Task03_Liver', 
#              '../../data/Task07_Pancreas',
#              '../../data/Task08_HepaticVessel']

# save_dirs = ['./Spleen/data', 
#              './Liver/data', 
#              './Pancreas/data', 
#              './HepaticVessel/data']

data_dirs = ['../../data/Task03_Liver', 
             '../../data/Task07_Pancreas',
             '../../data/Task08_HepaticVessel']

save_dirs = ['./Liver/data', 
             './Pancreas/data', 
             './HepaticVessel/data']

for i, data_dir in enumerate(data_dirs):
    if not osp.exists(save_dirs[i]):
        os.mkdir(save_dirs[i])

    img_paths = glob(osp.join(data_dir, 'imagesTr_res') + '/*.*')

    with tqdm(total=len(img_paths)) as pbar:
        for img_path in img_paths:
            msk_path = osp.join(data_dir, 'labelsTr_res', osp.basename(img_path))
            img = nib.load(img_path)
            specs = img.affine
            img = img.get_fdata()
            msk = nib.load(msk_path).get_fdata()
            npz_save_path = osp.join(save_dirs[i], f'{osp.basename(img_path)[:-7]}.npz')
            np.savez(npz_save_path, data_image=img, data_mask=msk, data_specs=specs)
 
            pbar.update(1)
    

100%|█████████████████████████████████████████████| 303/303 [09:07<00:00,  1.81s/it]
